Name: Maria Miliou \
Course: Machine Learning
# Assignment 3 


Save data to your drive and mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Feedforward Neural Network**


###**Step 1**: Loading data (mfccs)




In [ ]:
import numpy as np
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader #. It represents a Python iterable over a dataset, with support for


# Dictionary for mapping between name of classes and integers 0-3
map_labels = {"blues": 0, "classical": 1, "hiphop": 2, "rock_metal_hardrock": 3}

# Create custom dataset

class MfccDataset(Dataset):
    def __init__(self, labels_file, X_file, transform = None):
        labels = np.load(labels_file)
        self.labels = [map_labels[x] for x in labels]  # Transform labels from string to int
        self.X = np.load(X_file)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
      sample = {'X': self.X[idx], 'label' : self.labels[idx]}
      if self.transform:
        sample = self.transform(sample)
        
      return sample

# Define transform for transforming data from numpy arrays to tensor 
class ToTensor(object):
    def __call__(self, sample):
      X, label = sample['X'], sample['label']
      return { 'X': torch.from_numpy(X), 'label' : label}


In [ ]:
# Train, validation and test datasets

train_dataset = MfccDataset(
    labels_file="/content/drive/MyDrive/data/music_genre_data_di/train/mfccs/labels.npy",
    X_file= "/content/drive/MyDrive/data/music_genre_data_di/train/mfccs/X.npy",
    transform = ToTensor())

val_dataset = MfccDataset(
    labels_file="/content/drive/MyDrive/data/music_genre_data_di/val/mfccs/labels.npy",
    X_file= "/content/drive/MyDrive/data/music_genre_data_di/val/mfccs/X.npy",
    transform = ToTensor())

test_dataset = MfccDataset(
    labels_file="/content/drive/MyDrive/data/music_genre_data_di/test/mfccs/labels.npy",
    X_file= "/content/drive/MyDrive/data/music_genre_data_di/test/mfccs/X.npy",
    transform = ToTensor())


In [ ]:
# Loading data
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)

###**Step 2**: Definition of Neural Network


In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(26, 128),
            #nn.ReLU(),
            nn.Linear(128, 32),
            #nn.ReLU(),
            nn.Linear(32, 4),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits



###**Step 3**: Definition of training function

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix

def train_loop(dataloader, model, loss_fn, optimizer, Nepochs):
  size = len(dataloader.dataset)
  for epoch in range(Nepochs):
      print(f"-------------------------------\nEpoch {epoch+1}\n-------------------------------")
      for batch, sample in enumerate(dataloader):
        X = sample['X']
        y = sample['label']

        X = X.to(device)
        y = y.to(device)

        # Compute prediction and loss
        pred = model(X.float())
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #per 50 batches report the value of the loss function on the training set
        if batch % 50 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

  return model

###**Step 4**: Definition of Testing function

In [ ]:

def test_fn(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct, f1 = 0, 0, 0

    # the no_grad command tells PyTorch that we dont want to accumulate gradients for these operations. Disabling gradient calculation is useful for inference with no backpropagation
    with torch.no_grad():
        for sample in dataloader:

            X = sample['X']
            y = sample['label']

            X = X.to(device)
            y = y.to(device)
            
            pred = model(X.float())
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            f1 += f1_score(y.to('cpu'), pred.to('cpu').argmax(1), average = 'macro')
            c_matrix = confusion_matrix(y.to('cpu'), pred.to('cpu').argmax(1))

    test_loss /= size
    correct /= size
    f1 /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}, Avg f1: {f1:>8f} \n")


### **Step 5**: Training NN

In [ ]:
import time

device = 'cpu'
model = NeuralNetwork().to(device)

learning_rate = 2e-3
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()
num_epochs = 30

print(f"Device {device}")

#Training
start_cpu = time.time()
train_loop(train_dataloader, model, loss_fn, optimizer, num_epochs)
end_cpu = time.time()

#Testing
test_fn(test_dataloader, model,loss_fn)

print(f"CPU time: {end_cpu - start_cpu}")

Device cpu
-------------------------------
Epoch 1
-------------------------------
loss: 1.460196  [    0/ 3200]
loss: 1.382018  [  800/ 3200]
loss: 1.377698  [ 1600/ 3200]
loss: 1.447004  [ 2400/ 3200]
-------------------------------
Epoch 2
-------------------------------
loss: 1.373881  [    0/ 3200]
loss: 1.388828  [  800/ 3200]
loss: 1.320949  [ 1600/ 3200]
loss: 1.410554  [ 2400/ 3200]
-------------------------------
Epoch 3
-------------------------------
loss: 1.427715  [    0/ 3200]
loss: 1.348772  [  800/ 3200]
loss: 1.399906  [ 1600/ 3200]
loss: 1.364457  [ 2400/ 3200]
-------------------------------
Epoch 4
-------------------------------
loss: 1.339833  [    0/ 3200]
loss: 1.434092  [  800/ 3200]
loss: 1.331324  [ 1600/ 3200]
loss: 1.395113  [ 2400/ 3200]
-------------------------------
Epoch 5
-------------------------------
loss: 1.407538  [    0/ 3200]
loss: 1.346398  [  800/ 3200]
loss: 1.351049  [ 1600/ 3200]
loss: 1.400627  [ 2400/ 3200]
-----------------------------

### **Step 6**: Taining NN using GPU



In [ ]:
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = NeuralNetwork().to(device)

learning_rate = 2e-3
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()
num_epochs = 30

print(f"Device {device}")

#Training
start_cuda = time.time()
train_loop(train_dataloader, model, loss_fn, optimizer, num_epochs)
end_cuda = time.time()

#Testing
test_fn(test_dataloader, model,loss_fn)

print(f"GPU time: {end_cuda - start_cuda}")

Device cuda
-------------------------------
Epoch 1
-------------------------------
loss: 2.157213  [    0/ 3200]
loss: 1.404286  [  800/ 3200]
loss: 1.377580  [ 1600/ 3200]
loss: 1.365701  [ 2400/ 3200]
-------------------------------
Epoch 2
-------------------------------
loss: 1.416197  [    0/ 3200]
loss: 1.359858  [  800/ 3200]
loss: 1.322867  [ 1600/ 3200]
loss: 1.405018  [ 2400/ 3200]
-------------------------------
Epoch 3
-------------------------------
loss: 1.297819  [    0/ 3200]
loss: 1.314305  [  800/ 3200]
loss: 1.303071  [ 1600/ 3200]
loss: 1.288286  [ 2400/ 3200]
-------------------------------
Epoch 4
-------------------------------
loss: 1.272789  [    0/ 3200]
loss: 1.294675  [  800/ 3200]
loss: 1.390621  [ 1600/ 3200]
loss: 1.278575  [ 2400/ 3200]
-------------------------------
Epoch 5
-------------------------------
loss: 1.328136  [    0/ 3200]
loss: 1.319416  [  800/ 3200]
loss: 1.257396  [ 1600/ 3200]
loss: 1.332420  [ 2400/ 3200]
----------------------------

Learning's execution time in both GPU and CPU

In [ ]:
print(f"CPU time: {end_cpu - start_cpu}")
print(f"GPU time: {end_cuda - start_cuda}")

CPU time: 3.6981163024902344
GPU time: 6.417418956756592


### **Step 7**: Choosing "best" model


Editing the training procedure to evaluate model's snapshots with validation set.

In [ ]:
import copy

def train_loop_val(dataloader, model, loss_fn, optimizer, Nepochs):
  size = len(dataloader.dataset)

  max_val_f1 = 0.0

  for epoch in range(Nepochs):

      for batch, sample in enumerate(dataloader):
        X = sample['X']
        y = sample['label']

        X = X.to(device)
        y = y.to(device)

        # Compute prediction and loss
        pred = model(X.float())
        loss = loss_fn(pred, y) 

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

      # Compute f1 score of validation set in this model
      val_f1 = 0.0
      for batch, sample in enumerate(val_dataloader):
          X = sample['X']
          y = sample['label']

          X = X.to(device)
          y = y.to(device)

          # Compute prediction
          pred = model(X.float())
          val_f1 += f1_score(y.cpu(), pred.cpu().argmax(1), average = 'macro')

      if val_f1 > max_val_f1:
        max_val_f1 = val_f1
        print(f"Found better model in epoch {epoch}! Saving... ")
        torch.save(model.state_dict(), 'saved_model.pth')
        

  return model

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = NeuralNetwork().to(device)

train_loop_val(train_dataloader, model, loss_fn, optimizer, num_epochs)

best_model = NeuralNetwork().to(device)
best_model.load_state_dict(torch.load('saved_model.pth'))
best_model.eval()
test_fn(test_dataloader, best_model ,loss_fn)

Found better model in epoch 0! Saving... 
Found better model in epoch 1! Saving... 
Found better model in epoch 5! Saving... 
Test Error: 
 Accuracy: 23.8%, Avg loss: 0.092601, Avg f1: 0.010153 



# **Convolutional Neural Network**

###**Step 1**: Loading data (spectograms)




In [ ]:
import numpy as np
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader

In [ ]:

# Create custom dataset

class MelgramDataset(Dataset):
    def __init__(self, labels_file, X_file, transform = None):
        labels = np.load(labels_file)
        self.labels = [map_labels[x] for x in labels]  # Transform labels from string to int
        self.X = np.load(X_file)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
      sample = {'X': self.X[idx], 'label' : self.labels[idx]}
      if self.transform:
        sample = self.transform(sample)
        
      return sample

# Define transform for transforming data from numpy arrays to tensor 
class ToTensor(object):
    def __call__(self, sample):
      X, label = sample['X'], sample['label']
      return { 'X': torch.from_numpy(X), 'label' : label}


In [ ]:
# Train, validation and test datasets

train_dataset = MelgramDataset(
    labels_file="/content/drive/MyDrive/data/music_genre_data_di/train/melgrams/labels.npy",
    X_file= "/content/drive/MyDrive/data/music_genre_data_di/train/melgrams/X.npy",
    transform = ToTensor())

val_dataset = MelgramDataset(
    labels_file="/content/drive/MyDrive/data/music_genre_data_di/val/melgrams/labels.npy",
    X_file= "/content/drive/MyDrive/data/music_genre_data_di/val/melgrams/X.npy",
    transform = ToTensor())

test_dataset = MelgramDataset(
    labels_file="/content/drive/MyDrive/data/music_genre_data_di/test/melgrams/labels.npy",
    X_file= "/content/drive/MyDrive/data/music_genre_data_di/test/melgrams/X.npy",
    transform = ToTensor())


In [ ]:
# Loading data
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)

###**Step 2**: Define CNN

In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 

class CNNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 16, 5)
    self.conv2 = nn.Conv2d(16, 32, 5)
    self.conv3 = nn.Conv2d(32, 64, 5)
    self.conv4 = nn.Conv2d(64, 128, 5)

    self.dense1 = nn.Linear(128*5*112, 1024)
    self.dense2 = nn.Linear(1024, 256 )
    self.dense3 = nn.Linear(256, 32)
    self.dense4 = nn.Linear(32, 4)

  def forward(self, x):
      x = x.unsqueeze(1)
      x = x.view(-1, 128*5*112)
      x = self.dense1(x)
      x = self.dense2(x)
      x = self.dense3(x)
      x = self.dense4(x)

      return x 


device = 'cuda' if torch.cuda.is_available() else 'cpu'
net= CNNet().to(device)

###**Step 3**: Network training





In [ ]:
learning_rate = 2e-3
optimizer = torch.optim.SGD(net.parameters(), lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()
num_epochs = 30

print(f"Device {device}")

train_loop_val(train_dataloader, net, loss_fn, optimizer, num_epochs)

best_model = CNNet()
best_model.load_state_dict(torch.load('saved_model1.pth'))
best_model.eval()
test_fn(test_dataloader, best_model ,loss_fn)

Device cpu


RuntimeError: ignored

Network can not be trained because of high dimensionality.

###**Step 4**: Pooling and padding

In [ ]:

class CNNet_pp(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 16, 5, padding=2)
    self.conv2 = nn.Conv2d(16, 32, 5, padding=2)
    self.conv3 = nn.Conv2d(32, 64, 5, padding=2)
    self.conv4 = nn.Conv2d(64, 128, 5, padding=2)

    self.dense1 = nn.Linear(1024, 1024)
    self.dense2 = nn.Linear(1024, 256 )
    self.dense3 = nn.Linear(256, 32)
    self.dense4 = nn.Linear(32, 4)

  def forward(self, x):
      x = x.unsqueeze(1)
      x = F.max_pool2d(self.conv1(x), kernel_size=2)
      x = F.max_pool2d(self.conv2(x), kernel_size=2)
      x = F.max_pool2d(self.conv3(x), kernel_size=2)
      x = F.max_pool2d(self.conv4(x), kernel_size=2)
      x = x.view(-1, 1024)
      x = self.dense1(x)
      x = self.dense2(x)
      x = self.dense3(x)
      x = self.dense4(x)

      return x 


device = 'cuda' if torch.cuda.is_available() else 'cpu'
net_pp= CNNet_pp().to(device)


In [ ]:

learning_rate = 2e-3
optimizer = torch.optim.SGD(net_pp.parameters(), lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()
num_epochs = 30

print(f"Device {device}")

train_loop_val(train_dataloader, net_pp, loss_fn, optimizer, num_epochs)

best_model = CNNet_pp().to(device)
best_model.load_state_dict(torch.load('saved_model.pth'))
best_model.eval()
test_fn(test_dataloader, best_model ,loss_fn)

Device cpu
Found better model in epoch 0! Saving... 
Found better model in epoch 2! Saving... 
Found better model in epoch 4! Saving... 
Found better model in epoch 6! Saving... 
Found better model in epoch 7! Saving... 
Found better model in epoch 8! Saving... 
Found better model in epoch 9! Saving... 
Found better model in epoch 10! Saving... 
Found better model in epoch 11! Saving... 
Found better model in epoch 14! Saving... 
Test Error: 
 Accuracy: 70.9%, Avg loss: 0.046573, Avg f1: 0.023122 



###**Step 5**: Optimizers

How the use of different optimizers reflex to accuracy anf f1 score

|  | Adadelta|Adagrad |SGD | Adam
|----|-----|-----|----|-------
|Accuracy|73.1%|73.2%|70.9%|70.9%|
|f1| 0.025|0.026|0.023|0.022|

# **Ερώτημα 3** 
## Deep Learning

###**Step 1**: Reproducibility

In [ ]:
import numpy as np
import random, os

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

seed=0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
class CNNet3(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 16, 5, padding=2)
    self.conv2 = nn.Conv2d(16, 32, 5, padding=2)
    self.conv3 = nn.Conv2d(32, 64, 5, padding=2)
    self.conv4 = nn.Conv2d(64, 128, 5, padding=2)

    self.dense1 = nn.Linear(1024, 1024)
    self.dense2 = nn.Linear(1024, 256 )
    self.dense3 = nn.Linear(256, 32)
    self.dense4 = nn.Linear(32, 4)

  def forward(self, x):
      x = x.unsqueeze(1)
      x = F.max_pool2d(self.conv1(x), kernel_size=2)
      x = F.max_pool2d(self.conv2(x), kernel_size=2)
      x = F.max_pool2d(self.conv3(x), kernel_size=2)
      x = F.max_pool2d(self.conv4(x), kernel_size=2)
      x = x.view(-1, 1024)
      x = self.dense1(x)
      x = self.dense2(x)
      x = self.dense3(x)
      x = self.dense4(x)

      return x 

In [ ]:

net_seed = CNNet3().to(device)
#Training
train_loop_val(train_dataloader, net_seed, loss_fn, optimizer, num_epochs)

#Take the best model according to validation set
best_model = CNNet3().to(device)
best_model.load_state_dict(torch.load('saved_model.pth'))
best_model.eval()

#Testing
test_fn(test_dataloader, best_model ,loss_fn)

Found better model in epoch 0! Saving... 
Found better model in epoch 1! Saving... 
Found better model in epoch 4! Saving... 
Test Error: 
 Accuracy: 23.6%, Avg loss: 0.088664, Avg f1: 0.014071 



###**Step 2**: Activation functions

In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 

class AF_CNNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 16, 5, padding=2)
    self.conv2 = nn.Conv2d(16, 32, 5, padding=2)
    self.conv3 = nn.Conv2d(32, 64, 5, padding=2)
    self.conv4 = nn.Conv2d(64, 128, 5, padding=2)

    self.dense1 = nn.Linear(1024, 1024)
    self.dense2 = nn.Linear(1024, 256 )
    self.dense3 = nn.Linear(256, 32)
    self.dense4 = nn.Linear(32, 4)

  def forward(self, x):
      x = x.unsqueeze(1)
      x = F.max_pool2d(F.logsigmoid(self.conv1(x)), kernel_size=2)
      x = F.max_pool2d(F.logsigmoid(self.conv2(x)), kernel_size=2)
      x = F.max_pool2d(F.logsigmoid(self.conv3(x)), kernel_size=2)
      x = F.max_pool2d(F.logsigmoid(self.conv4(x)), kernel_size=2)
      x = x.view(-1, 1024)
      x = F.logsigmoid(self.dense1(x))
      x = F.logsigmoid(self.dense2(x))
      x = F.logsigmoid(self.dense3(x))
      x = self.dense4(x)

      return x 


device = 'cuda' if torch.cuda.is_available() else 'cpu'
net_af= AF_CNNet().to(device)

In [ ]:
learning_rate = 2e-3
optimizer = torch.optim.SGD(net_af.parameters(), lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()
num_epochs = 30

print(f"Device {device}")

train_loop_val(train_dataloader, net_af, loss_fn, optimizer, num_epochs)

best_model = AF_CNNet().to(device)
best_model.load_state_dict(torch.load('saved_model.pth'))
best_model.eval()
test_fn(test_dataloader, best_model ,loss_fn)

Device cpu
Found better model in epoch 0! Saving... 
Found better model in epoch 1! Saving... 
Found better model in epoch 7! Saving... 
Found better model in epoch 19! Saving... 
Found better model in epoch 22! Saving... 
Found better model in epoch 25! Saving... 
Found better model in epoch 26! Saving... 
Found better model in epoch 29! Saving... 
Test Error: 
 Accuracy: 61.5%, Avg loss: 0.058939, Avg f1: 0.018035 



|  | ReLU|LogSigmoid |ELU| 
|----|-----|-----|----
|Accuracy|67.8%|61.5%|73.4%|
|f1| 0.021|0.018|0.046|

###**Step 4**: Learning rate scheduler

In [ ]:
def train_sch(dataloader, model, loss_fn, optimizer, Nepochs, scheduler):
  size = len(dataloader.dataset)

  max_val_f1 = 0.0

  for epoch in range(Nepochs):

      for batch, sample in enumerate(dataloader):
        X = sample['X']
        y = sample['label']

        X = X.to(device)
        y = y.to(device)

        # Compute prediction and loss
        pred = model(X.float())
        loss = loss_fn(pred, y) 

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
      

      # Compute f1 score of validation set in this model
      val_f1 = 0.0
      for batch, sample in enumerate(val_dataloader):
          X = sample['X']
          y = sample['label']

          X = X.to(device)
          y = y.to(device)

          # Compute prediction
          pred = model(X.float())
          val_f1 += f1_score(y.cpu(), pred.cpu().argmax(1), average = 'macro')

      if val_f1 > max_val_f1:
        max_val_f1 = val_f1
        print(f"Found better model in epoch {epoch}! Saving... ")
        torch.save(model.state_dict(), 'saved_model.pth')

      scheduler.step() 

  return model

In [ ]:
import torch.optim.lr_scheduler

learning_rate = 2e-3
optimizer = torch.optim.SGD(net_af.parameters(), lr = learning_rate)
scheduler =  torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9 ,verbose=True)
loss_fn = nn.CrossEntropyLoss()
num_epochs = 30

print(f"Device {device}")

train_sch(train_dataloader, net_af, loss_fn, optimizer, num_epochs, scheduler)

best_model = AF_CNNet().to(device)
best_model.load_state_dict(torch.load('saved_model.pth'))
best_model.eval()
test_fn(test_dataloader, best_model ,loss_fn)

Adjusting learning rate of group 0 to 2.0000e-03.
Device cpu
Found better model in epoch 0! Saving... 
Adjusting learning rate of group 0 to 1.8000e-03.
Adjusting learning rate of group 0 to 1.6200e-03.
Found better model in epoch 2! Saving... 
Adjusting learning rate of group 0 to 1.4580e-03.
Adjusting learning rate of group 0 to 1.3122e-03.
Adjusting learning rate of group 0 to 1.1810e-03.
Found better model in epoch 5! Saving... 
Adjusting learning rate of group 0 to 1.0629e-03.
Adjusting learning rate of group 0 to 9.5659e-04.
Adjusting learning rate of group 0 to 8.6093e-04.
Adjusting learning rate of group 0 to 7.7484e-04.
Adjusting learning rate of group 0 to 6.9736e-04.
Found better model in epoch 10! Saving... 
Adjusting learning rate of group 0 to 6.2762e-04.
Adjusting learning rate of group 0 to 5.6486e-04.
Adjusting learning rate of group 0 to 5.0837e-04.
Adjusting learning rate of group 0 to 4.5754e-04.
Adjusting learning rate of group 0 to 4.1178e-04.
Adjusting learning r

###**Step 5**: Batch Normalization


In [ ]:
class BN_CNNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 16, 5, padding=2)
    self.b1 = nn.BatchNorm2d(16)
    self.conv2 = nn.Conv2d(16, 32, 5, padding=2)
    self.b2 = nn.BatchNorm2d(32)
    self.conv3 = nn.Conv2d(32, 64, 5, padding=2)
    self.b3 = nn.BatchNorm2d(64)
    self.conv4 = nn.Conv2d(64, 128, 5, padding=2)
    self.b4 = nn.BatchNorm2d(128)

    self.dense1 = nn.Linear(1024, 1024)
    self.dense2 = nn.Linear(1024, 256)
    self.dense3 = nn.Linear(256, 32)
    self.dense4 = nn.Linear(32, 4)

  def forward(self, x):
      x = x.unsqueeze(1)
      x = F.max_pool2d(F.relu(self.b1(self.conv1(x))), kernel_size=2)
      x = F.max_pool2d(F.relu(self.b2(self.conv2(x))), kernel_size=2)
      x = F.max_pool2d(F.relu(self.b3(self.conv3(x))), kernel_size=2)
      x = F.max_pool2d(F.relu(self.b4(self.conv4(x))), kernel_size=2)
      x = x.view(-1, 1024)

      x = F.relu(self.dense1(x))
      x = F.relu(self.dense2(x))
      x = F.relu(self.dense3(x))
      x = self.dense4(x)

      return x 

device = 'cuda' if torch.cuda.is_available() else 'cpu'
net_bn= BN_CNNet().to(device)

In [ ]:
train_sch(train_dataloader, net_bn, loss_fn, optimizer, num_epochs, scheduler)

best_model = BN_CNNet().to(device)
best_model.load_state_dict(torch.load('saved_model.pth'))
best_model.eval()
test_fn(test_dataloader, best_model ,loss_fn)

Found better model in epoch 0! Saving... 
Adjusting learning rate of group 0 to 7.6304e-05.
Found better model in epoch 1! Saving... 
Adjusting learning rate of group 0 to 6.8674e-05.
Adjusting learning rate of group 0 to 6.1806e-05.
Found better model in epoch 3! Saving... 
Adjusting learning rate of group 0 to 5.5626e-05.
Adjusting learning rate of group 0 to 5.0063e-05.
Adjusting learning rate of group 0 to 4.5057e-05.
Adjusting learning rate of group 0 to 4.0551e-05.
Adjusting learning rate of group 0 to 3.6496e-05.
Adjusting learning rate of group 0 to 3.2846e-05.
Adjusting learning rate of group 0 to 2.9562e-05.
Adjusting learning rate of group 0 to 2.6606e-05.
Adjusting learning rate of group 0 to 2.3945e-05.
Adjusting learning rate of group 0 to 2.1551e-05.
Adjusting learning rate of group 0 to 1.9395e-05.
Adjusting learning rate of group 0 to 1.7456e-05.
Adjusting learning rate of group 0 to 1.5710e-05.
Adjusting learning rate of group 0 to 1.4139e-05.
Adjusting learning rate 